In [14]:
import numpy as np
np.set_printoptions(threshold=10000, suppress=True)
import pandas as pd
import matplotlib.pyplot as plt

# PARTIE 1

## Chargement et visualisation des données

In [ ]:
mickey = np.loadtxt('mouse.txt')
x1 = mickey[:,0]
x2 = mickey[:,1]

plt.scatter(x1, x2)
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('Mouse data')
plt.show()

## Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest

iforest = IsolationForest(n_estimators=100, max_samples='auto')
iforest.fit(mickey)

df = pd.DataFrame(mickey)
df['scores'] = iforest.decision_function(mickey)
# -1 for outliers, 1 for inliers
df['anomaly'] = iforest.predict(mickey)
df.head(20)

In [ ]:
anomaly = df.loc[df['anomaly'] == -1]
anomaly_index = list(anomaly.index)
print(len(anomaly))

## Local Outlier Factor

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

lof = LocalOutlierFactor(n_neighbors=20, contamination=0.1)
pred = lof.fit_predict(mickey)

# -1 for outliers, 1 for inliers
print(pred)


## Mieux choisir le seuil de contamination

### Méthode 1 : Visualisation des anomalies - Isolation Forest

Pour cette méthode, nous allons visualiser l'histogramme des scores de l'Isolation Forest. Nous allons ensuite calculer le seuil de contamination optimal.

In [ ]:
scrores = df['scores']

# récupérer les informations de l'histogramme
frequencies, bins = np.histogram(scrores, bins=100)

# calculer la pente de chaque barre
bin_diff = np.diff(bins)
slope = np.diff(frequencies) / bin_diff[:-1]

# trouver le seuil
threshold_index = np.where(slope < -np.abs(slope).max()*0.09)[0][0]
threshold = bins[threshold_index +1]

plt.hist(scrores, bins=100)
plt.axvline(threshold, color='red', linestyle='--')


In [ ]:
nombre_sous_seuil = (df['scores'] < threshold).sum()
print("Pourcentage d'erreur : ",nombre_sous_seuil*100/len(df), '%')

### Méthode 2 : Calculer le seuil avec un Z-score - Local Outlier Factor

Pour cette méthode, nous allons calculer le seuil de contamination optimal en utilisant un Z-score.

In [ ]:
lof_scores = lof.negative_outlier_factor_
# Calcul du seuil avec Z-score
threshold = np.mean(lof_scores) - 3 * np.std(lof_scores)

print("Seuil LOF : ", threshold)
nombre_sous_seuil = (lof_scores < threshold).sum()
print("Pourcentage d'erreur LOF : ",nombre_sous_seuil*100/len(df), '%')

## Visualisation des données aberrantes

In [ ]:
iforest = IsolationForest(n_estimators=100, max_samples='auto', contamination=0.036)
iforest.fit(mickey)
df['scores'] = iforest.decision_function(mickey)
df['anomaly'] = iforest.predict(mickey)
plt.scatter(x1, x2, c=df['anomaly'], cmap='coolwarm', s=20, edgecolors='k')
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('Anomalies detected by Isolation Forest')
plt.show()

In [ ]:
# print les index des anomalies
anomaly = df.loc[df['anomaly'] == -1]
anomaly_index = list(anomaly.index)
print(anomaly_index)

In [ ]:
lof = LocalOutlierFactor(n_neighbors=20, contamination=0.016)
pred = lof.fit_predict(mickey)
plt.scatter(x1, x2, c=pred, cmap='coolwarm', s=20, edgecolors='k')
plt.xlabel('x1')
plt.ylabel('x2')
plt.title('Anomalies detected by Local Outlier Factor')
plt.show()

In [ ]:
# print les index des anomalies
anomaly = df.loc[pred == -1]
anomaly_index = list(anomaly.index)
print(anomaly_index)

## Comparaison des méthodes

La méthode Isolation Forest avec un seuil de contamination calculé grâce à l'histogramme, permet de détecter toutes les anomalies. Cepedant, elle détecter aussi des points comme étant anormal alors qu'il ne le sont pas. D'un autre côté, la méthode Local Outlier Factor avec Z-scores permet de détecter les anomalies mais ne les trouve pas toutes. Aucune méthode n'est meilleur que l'autre, cela dépend si l'on veut détecter toutes les anomalies ou si l'on veut éviter de faux positifs.

# PARTIE 2

## Préparation des données

In [ ]:
from sklearn.preprocessing import StandardScaler

credit_card = pd.read_csv('creditcard.csv')
credit_card = credit_card.drop(columns=['Time'])

# Normalisation
scaler = StandardScaler()
credit_card['Amount'] = scaler.fit_transform(credit_card['Amount'].values.reshape(-1,1))

credit_card.head()

## UnderSampling

## OverSampling

## Balancing

## Isolation Forest

## Local Outlier Factor